In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from google.colab import drive
import yfinance as yf
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Flatten, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Volatility/New_data_2000_2024/Not_normal_wd/'

Mounted at /content/drive


In [ ]:
def set_index(df):
  df.index = pd.to_datetime(df['Date'])
  df.drop(columns=['Date'], inplace=True)

In [ ]:
filename = 'GKYZV_data.csv'
HV_data = pd.read_csv(path + filename)

In [ ]:
set_index(HV_data)

In [ ]:
tm_data = HV_data

In [ ]:
HV_data = HV_data.drop(columns=['GKYZV'])

In [ ]:
HV_data['GKYZV'] = tm_data['GKYZV']

In [ ]:
HV_data

,"('FIGARCH', 'normal', 0)","('FIGARCH', 'normal', 1)","('FIGARCH', 'gaussian', 0)","('FIGARCH', 'gaussian', 1)",gdp_growth,volume,"('GARCH', 'normal', 0)","('GARCH', 'normal', 1)","('GARCH', 'gaussian', 0)","('GARCH', 'gaussian', 1)",...,usd_eur,usd_jpy,usd_gbp,usd_cny,usd_cad,usd_mxn,gt_data,log_returns,"('APARCH', 'studentst', 1)",GKYZV
Date,,,,,,,,,,,,,,,,,,,,,
2000-02-02,0.038388,0.038388,0.038388,0.038388,28984.929215,2.945453e+09,0.039601,0.038301,0.039601,0.038301,...,2.781970,307.252039,4.529620,23.415205,4.090860,26.750733,209.303607,-0.005604,0.037128,0.034017
2000-02-03,0.038633,0.038633,0.038633,0.038633,28984.929215,2.935907e+09,0.037979,0.038355,0.037979,0.038355,...,2.799436,311.491144,4.529266,23.412659,4.119569,26.561052,209.303607,-0.008693,0.038336,0.034689
2000-02-04,0.036990,0.036990,0.036990,0.036990,28984.929215,3.274647e+09,0.036791,0.037755,0.036791,0.037755,...,2.728548,306.262089,4.474254,23.415452,4.106629,26.433950,209.303607,0.000572,0.049778,0.032966
2000-02-07,0.046474,0.046474,0.046474,0.046474,28984.929215,3.413593e+09,0.046593,0.055072,0.046593,0.055072,...,2.733357,299.491542,4.460147,23.414391,4.140641,26.339020,209.303607,0.027307,0.065732,0.036397
2000-02-08,0.046711,0.046711,0.046711,0.046711,28984.929215,2.868803e+09,0.049131,0.041984,0.049131,0.041984,...,2.730104,301.761354,4.470789,23.415982,4.142338,26.135551,209.303607,0.010478,0.043373,0.037620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-26,0.000121,0.000121,0.000121,0.000121,0.000000,1.609587e+08,0.000120,-0.000665,0.000120,-0.000665,...,-0.002546,0.509117,-0.003041,0.005162,0.001980,-0.105925,0.404061,0.008246,-0.004076,0.000040
2024-08-27,-0.004368,-0.004368,-0.004368,-0.004368,0.000000,9.767973e+07,-0.003889,-0.004798,-0.003889,-0.004798,...,0.002051,-0.551543,-0.001202,-0.006788,-0.001556,-0.173948,0.404061,0.009336,-0.005604,0.000047
2024-08-28,0.000483,0.000483,0.000483,0.000483,0.000000,2.498703e+08,0.000280,-0.001148,0.000280,-0.001148,...,0.000849,0.247487,0.000849,0.002263,0.002051,-0.219910,0.707107,0.010307,-0.001642,0.000051


# Split train, validation and test data

In [ ]:
def split_data(data, target, train_end =datetime(2022, 5, 30), test_start=datetime(2022, 5, 31), test_size=0.1):
  test_data = data.loc[test_start:]

  train_val = data.loc[:train_end]
  train_data, val_data = train_test_split(train_val, test_size=test_size, shuffle=False)

  return train_data, val_data, test_data

In [ ]:
def sliding_window_transform(X, y, timesteps):
  X_windows = []
  y_windows = []
  for i in range(len(X) - timesteps + 1):
    X_window = X[i:i + timesteps]
    y_window = y[i + timesteps - 1]
    X_windows.append(X_window)
    y_windows.append(y_window)

  return np.array(X_windows), np.array(y_windows)

In [ ]:
# Define the rolling window function
def rolling_window(df, in_sample_window_size, out_of_sample_size):
  X, y = [], []
  for i in range(in_sample_window_size, len(df) - out_of_sample_size):
    X.append(df.iloc[i - in_sample_window_size:i, :-1].values)  # All features except the target column
    y.append(df.iloc[i:i + out_of_sample_size, -1].values)  # Target column

  return np.array(X), np.array(y)

# Model

In [ ]:
def calculate_mape(actual, predicted):
  actual, predicted = np.array(actual), np.array(predicted)
  return np.mean(np.abs((actual - predicted) / actual)) * 100

## GKYZV

In [ ]:
daily_train, daily_val, daily_test = split_data(HV_data, 'GKYZV')

In [ ]:
in_sam_win_sz = 25
out_sam_win_sz = 5

In [ ]:
HV_X_train, HV_y_train = rolling_window(daily_train, in_sam_win_sz, out_sam_win_sz)
HV_X_val, HV_y_val = rolling_window(daily_val, in_sam_win_sz, out_sam_win_sz)
HV_X_test, HV_y_test = rolling_window(daily_test, in_sam_win_sz, out_sam_win_sz)

In [ ]:
# HV_X_train = HV_X_train.reshape(HV_X_train.shape[0], win_sz, -1)
# HV_X_val = HV_X_val.reshape(HV_X_val.shape[0], win_sz, -1)
# HV_X_test = HV_X_test.reshape(HV_X_test.shape[0], win_sz, -1)

### LSTM

In [ ]:
lstm_garch_model = Sequential()
lstm_garch_model.add(LSTM(48, input_shape=(HV_X_train.shape[1], HV_X_train.shape[2]), return_sequences=True))
lstm_garch_model.add(Dropout(0.1))
lstm_garch_model.add(LSTM(16))
lstm_garch_model.add(Dropout(0.1))
lstm_garch_model.add(Dense(5))

lstm_garch_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
lstm_garch_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 25, 48)              │          14,976 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 16)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,221 (75.08 KB)

 Trainable params: 19,221 (75.08 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
lstm_history = lstm_garch_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - loss: 0.0122 - val_loss: 1.6761e-05
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - loss: 1.9743e-04 - val_loss: 4.8173e-06
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 9.5322e-05 - val_loss: 4.5335e-06
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 7.1378e-05 - val_loss: 2.8488e-06
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - loss: 5.5747e-05 - val_loss: 2.6069e-06
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - loss: 5.2924e-05 - val_loss: 3.0129e-06
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 4.0556e-05 - val_loss: 1.1291e-05
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 4.9074e-05 - val_loss: 1.7602e-06
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - loss: 4.4191e-05 - val_loss: 2.2289e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 3.4060e-05 - val_loss: 2.4687e-06
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step

In [ ]:
# Predictions
HV_test_predictions = lstm_garch_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step


In [ ]:
mse = mean_squared_error(HV_y_test, HV_test_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_test_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_test_predictions)
print(f'MAPE: {mape}%')

MSE: 3.7799650945069476e-06
RMSE: 0.0019442132327774512
MAE: 0.001746345970562458
MAPE: 9742.62088252184%


### GRU

In [ ]:
gru_model = Sequential()
gru_model.add(GRU(48, input_shape=(HV_X_train.shape[1], HV_X_train.shape[2]), return_sequences=True))
gru_model.add(Dropout(0.1))
gru_model.add(GRU(16))
gru_model.add(Dropout(0.1))
gru_model.add(Dense(5))

gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
gru_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru (GRU)                            │ (None, 25, 48)              │          11,376 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 16)                  │           3,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,629 (57.14 KB)

 Trainable params: 14,629 (57.14 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
gru_history = gru_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 29ms/step - loss: 0.0663 - val_loss: 1.5038e-04
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - loss: 4.3310e-04 - val_loss: 3.2675e-05
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms/step - loss: 1.6952e-04 - val_loss: 1.9553e-05
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 29ms/step - loss: 1.1767e-04 - val_loss: 1.2643e-05
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 38ms/step - loss: 7.9293e-05 - val_loss: 8.1973e-06
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 6.7210e-05 - val_loss: 7.1869e-06
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - loss: 6.2383e-05 - val_loss: 5.8415e-06
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - loss: 6.0973e-05 - val_loss: 3.8962e-06
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - loss: 5.6517e-05 - val_loss: 4.2629e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - loss: 4.3716e-05 - val_loss: 3.1495e-06
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step

In [ ]:
# Predictions
HV_gru_predictions = gru_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step


In [ ]:
mse = mean_squared_error(HV_y_test, HV_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 1.3310688057251731e-06
RMSE: 0.001153719552458557
MAE: 0.0008985396597258968
MAPE: 4684.324433871822%


### LSTM_GRU

In [ ]:
lstm_gru_model = Sequential()
lstm_gru_model.add(LSTM(48, input_shape=(HV_X_train.shape[1], HV_X_train.shape[2]), return_sequences=True))
lstm_gru_model.add(Dropout(0.1))
lstm_gru_model.add(GRU(16))
lstm_gru_model.add(Dropout(0.1))
lstm_gru_model.add(Dense(5))

lstm_gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
lstm_gru_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                        │ (None, 25, 48)              │          14,976 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_2 (GRU)                          │ (None, 16)                  │           3,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 18,229 (71.21 KB)

 Trainable params: 18,229 (71.21 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
lstm_gru_history = lstm_gru_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 12s 31ms/step - loss: 0.0136 - val_loss: 4.1639e-05
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 3.0020e-04 - val_loss: 1.4275e-05
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - loss: 1.4704e-04 - val_loss: 3.5828e-05
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - loss: 1.0734e-04 - val_loss: 7.9099e-06
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - loss: 7.0817e-05 - val_loss: 1.5164e-05
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - loss: 6.0907e-05 - val_loss: 4.6214e-06
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - loss: 5.3913e-05 - val_loss: 9.8891e-06
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - loss: 5.6886e-05 - val_loss: 2.1722e-05
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 5.4490e-05 - val_loss: 4.2369e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - loss: 3.8098e-05 - val_loss: 3.6276e-06
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/ste

In [ ]:
# Predictions
HV_lstm_gru_predictions = lstm_gru_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step


In [ ]:
mse = mean_squared_error(HV_y_test, HV_lstm_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_lstm_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_lstm_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 2.7903487091519282e-06
RMSE: 0.0016704336889418652
MAE: 0.0015434081722152501
MAPE: 8678.848685315423%


### BiLSTM

In [ ]:
bilstm_model = Sequential()
bilstm_model.add(Bidirectional(LSTM(48, return_sequences=True), input_shape=(HV_X_train.shape[1], HV_X_train.shape[2])))
bilstm_model.add(Dropout(0.1))
bilstm_model.add(Bidirectional(LSTM(16)))
bilstm_model.add(Dropout(0.1))
bilstm_model.add(Dense(5))

bilstm_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
bilstm_model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)        │ (None, 25, 96)              │          29,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 25, 96)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 32)                  │          14,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 5)                   │             165 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 44,581 (174.14 KB)

 Trainable params: 44,581 (174.14 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
bilstm_history = bilstm_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 15s 44ms/step - loss: 0.0171 - val_loss: 1.3172e-05
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 12s 54ms/step - loss: 1.1868e-04 - val_loss: 6.6702e-06
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 37ms/step - loss: 7.2998e-05 - val_loss: 5.4763e-06
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 10s 37ms/step - loss: 6.3550e-05 - val_loss: 4.2519e-06
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 12s 46ms/step - loss: 4.7881e-05 - val_loss: 6.0648e-06
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 12s 58ms/step - loss: 4.3888e-05 - val_loss: 3.3167e-06
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - loss: 5.1738e-05 - val_loss: 2.0216e-06
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 12s 53ms/step - loss: 3.8289e-05 - val_loss: 4.2976e-06
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 37ms/step - loss: 3.8243e-05 - val_loss: 2.7338e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 53ms/step - loss: 3.3132e-05 - val_loss: 4.0585e-06
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 38m

In [ ]:
# Predictions
HV_bilstm_predictions = bilstm_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step


In [ ]:
mse = mean_squared_error(HV_y_test, HV_bilstm_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_bilstm_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_bilstm_predictions)
print(f'MAPE: {mape}%')

MSE: 7.287767828248347e-06
RMSE: 0.00269958660321323
MAE: 0.0023142297591945663
MAPE: 12331.246729154558%


### BiLSTM_GRU

In [ ]:
bilstm_gru_model = Sequential()
bilstm_gru_model.add(Bidirectional(LSTM(48, return_sequences=True), input_shape=(HV_X_train.shape[1], HV_X_train.shape[2])))
bilstm_gru_model.add(Dropout(0.1))
bilstm_gru_model.add(GRU(16))
bilstm_gru_model.add(Dropout(0.1))
bilstm_gru_model.add(Dense(5))

bilstm_gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
bilstm_gru_model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_2 (Bidirectional)      │ (None, 25, 96)              │          29,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 25, 96)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ (None, 16)                  │           5,472 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 35,509 (138.71 KB)

 Trainable params: 35,509 (138.71 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
bilstm_gru_history = bilstm_gru_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 13s 45ms/step - loss: 0.0443 - val_loss: 1.3803e-04
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - loss: 7.1292e-04 - val_loss: 3.9578e-05
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 48ms/step - loss: 2.9739e-04 - val_loss: 2.1955e-05
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - loss: 1.6365e-04 - val_loss: 1.0674e-05
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 37ms/step - loss: 1.0599e-04 - val_loss: 1.0791e-05
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 32ms/step - loss: 9.7456e-05 - val_loss: 1.5324e-06
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 11s 34ms/step - loss: 6.2014e-05 - val_loss: 1.6448e-06
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 13s 49ms/step - loss: 5.3730e-05 - val_loss: 1.0495e-06
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - loss: 4.7837e-05 - val_loss: 1.2358e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 43ms/step - loss: 4.5378e-05 - val_loss: 1.6398e-06
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 32ms/s

In [ ]:
# Predictions
HV_bilstm_gru_predictions = bilstm_gru_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step


In [ ]:
mse = mean_squared_error(HV_y_test, HV_bilstm_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_bilstm_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_bilstm_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 2.1279771825544544e-06
RMSE: 0.0014587587814832356
MAE: 0.0013498935572735355
MAPE: 7232.559737756878%


In [ ]:
# path_1 = '/content/drive/MyDrive/Volatility/New_data_2000_2024/New_predictions/GKYZV/'

# df_pred = pd.DataFrame(HV_test_predictions[:,0], columns=['lstm_pred'])
# filename = 'LSTM_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_gru_predictions[:,0], columns=['gru_pred'])
# filename = 'GRU_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_lstm_gru_predictions[:,0], columns=['lstm_gru_pred'])
# filename = 'LSTM_GRU_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_bilstm_predictions[:,0], columns=['bilstm_pred'])
# filename = 'BiLSTM_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_bilstm_gru_predictions[:,0], columns=['bilstm_gru_pred'])
# filename = 'BiLSTM_GRU_pred.csv'
# df_pred.to_csv(path_1 + filename)